In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import random
import copy

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:

# Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)
SOS_token = 0
EOS_token = 1
hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
english_alphabets = [chr(alpha) for alpha in range(97, 123)]
hindi_alphabet_size = len(hindi_alphabets)
english_alphabet_size = len(english_alphabets)
hindi_alpha2index = {"SOS": 0,"EOS": 1}
english_alpha2index = {"SOS": 0,"EOS": 1}
for index, alpha in enumerate(hindi_alphabets):
    hindi_alpha2index[alpha] = index+2
for index, alpha in enumerate(english_alphabets):
    english_alpha2index[alpha] = index+2
hindi_index2alpha = {0 : "SOS", 1 : "EOS"}
english_index2alpha = { 0 : "SOS", 1 : "EOS"}
for index, alpha in enumerate(hindi_alphabets):
    hindi_index2alpha[index+2] = alpha
for index, alpha in enumerate(english_alphabets):
    english_index2alpha[index+2] = alpha 
print("Hindi A2I:\n", hindi_alpha2index)
print("-"*100)
print("English A2I:\n", english_alpha2index)
print("-"*100)
print("*"*100)
print("-"*100)
print("Hindi I2A:\n", hindi_index2alpha)
print("-"*100)
print("English I2A:\n", english_index2alpha)

Hindi A2I:
 {'SOS': 0, 'EOS': 1, 'ऀ': 2, 'ँ': 3, 'ं': 4, 'ः': 5, 'ऄ': 6, 'अ': 7, 'आ': 8, 'इ': 9, 'ई': 10, 'उ': 11, 'ऊ': 12, 'ऋ': 13, 'ऌ': 14, 'ऍ': 15, 'ऎ': 16, 'ए': 17, 'ऐ': 18, 'ऑ': 19, 'ऒ': 20, 'ओ': 21, 'औ': 22, 'क': 23, 'ख': 24, 'ग': 25, 'घ': 26, 'ङ': 27, 'च': 28, 'छ': 29, 'ज': 30, 'झ': 31, 'ञ': 32, 'ट': 33, 'ठ': 34, 'ड': 35, 'ढ': 36, 'ण': 37, 'त': 38, 'थ': 39, 'द': 40, 'ध': 41, 'न': 42, 'ऩ': 43, 'प': 44, 'फ': 45, 'ब': 46, 'भ': 47, 'म': 48, 'य': 49, 'र': 50, 'ऱ': 51, 'ल': 52, 'ळ': 53, 'ऴ': 54, 'व': 55, 'श': 56, 'ष': 57, 'स': 58, 'ह': 59, 'ऺ': 60, 'ऻ': 61, '़': 62, 'ऽ': 63, 'ा': 64, 'ि': 65, 'ी': 66, 'ु': 67, 'ू': 68, 'ृ': 69, 'ॄ': 70, 'ॅ': 71, 'ॆ': 72, 'े': 73, 'ै': 74, 'ॉ': 75, 'ॊ': 76, 'ो': 77, 'ौ': 78, '्': 79, 'ॎ': 80, 'ॏ': 81, 'ॐ': 82, '॑': 83, '॒': 84, '॓': 85, '॔': 86, 'ॕ': 87, 'ॖ': 88, 'ॗ': 89, 'क़': 90, 'ख़': 91, 'ग़': 92, 'ज़': 93, 'ड़': 94, 'ढ़': 95, 'फ़': 96, 'य़': 97, 'ॠ': 98, 'ॡ': 99, 'ॢ': 100, 'ॣ': 101, '।': 102, '॥': 103, '०': 104, '१': 105, '२': 106, '३': 107, '४': 108, '५'

In [4]:
data_train = pd.read_csv("hin_train.csv",header= None)
data_train = pd.DataFrame(np.array(data_train),columns=["English","Hindi"])
data_val = pd.read_csv("hin_valid.csv",header= None)
data_val = pd.DataFrame(np.array(data_val),columns=["English","Hindi"])
data_test = pd.read_csv("hin_test.csv",header= None)
data_test = pd.DataFrame(np.array(data_test),columns=["English","Hindi"])
print(data_train.shape,data_val.shape,data_test.shape)
data_train.head()

(51200, 2) (4096, 2) (4096, 2)


,English,Hindi
0,shastragaar,शस्त्रागार
1,bindhya,बिन्द्या
2,kirankant,किरणकांत
3,yagyopaveet,यज्ञोपवीत
4,ratania,रटानिया


In [5]:
data_train_X = np.array(data_train["English"])
data_train_y = np.array(data_train["Hindi"])
data_train_X,data_train_y

(array(['shastragaar', 'bindhya', 'kirankant', ..., 'asahmaton',
        'sulgaayin', 'anchuthengu'], dtype=object),
 array(['शस्त्रागार', 'बिन्द्या', 'किरणकांत', ..., 'असहमतों', 'सुलगायीं',
        'अंचुतेंगु'], dtype=object))

In [6]:
class Tokenize():
    def __init__(self,Lang_From,Lang_To):
        # Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)
        self.L1 = Lang_From
        self.L2 = Lang_To
        self.SOS_token = 0
        self.EOS_token = 1
        hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
        english_alphabets = [chr(alpha) for alpha in range(97, 123)]
        hindi_alphabet_size = len(hindi_alphabets)
        english_alphabet_size = len(english_alphabets)
        hindi_alpha2index = {"SOS": 0,"EOS": 1}
        english_alpha2index = {"SOS": 0,"EOS": 1}
        for index, alpha in enumerate(hindi_alphabets):
            hindi_alpha2index[alpha] = index+2
        for index, alpha in enumerate(english_alphabets):
            english_alpha2index[alpha] = index+2
        hindi_index2alpha = {0 : "SOS", 1 : "EOS"}
        english_index2alpha = { 0 : "SOS", 1 : "EOS"}
        for index, alpha in enumerate(hindi_alphabets):
            hindi_index2alpha[index+2] = alpha
        for index, alpha in enumerate(english_alphabets):
            english_index2alpha[index+2] = alpha 

        self.Lang_From_Alpha_2_Index = english_alpha2index
        self.Lang_To_Alpha_2_Index = hindi_alpha2index
        self.Lang_From_Index_2_Alpha = english_index2alpha
        self.Lang_To_Index_2_Alpha = hindi_index2alpha

    def tensorFromWord(self,Lang, word):
        if Lang == "L1":
            indexes = [self.Lang_From_Alpha_2_Index[letter] for letter in word]
        elif Lang == "L2":
            indexes = [self.SOS_token]+[self.Lang_To_Alpha_2_Index[letter] for letter in word]
        #print([self.EOS_token]*(30-len(indexes)))
        indexes+=[self.EOS_token]*(30-len(indexes))
        return torch.tensor(indexes, dtype=torch.long, device=device)#.view(-1, 1)

    def tensorsFromPair(self,pair):
        input_tensor = self.tensorFromWord("L1",pair[self.L1])
        target_tensor = self.tensorFromWord("L2",pair[self.L2])
        return (input_tensor, target_tensor)
    def tensorsFromData(self,Data):
        Tensors_Val = []
        for i in tqdm(range(Data.shape[0])):
            Tensors_Val.append(self.tensorsFromPair(Data.iloc[i]))
        return Tensors_Val
    def WordFromtensors(self,Lang, word):
        if Lang == "L1":
            letters = [self.Lang_From_Index_2_Alpha[letter.item()] for letter in word if ((letter.item() != EOS_token) and (letter.item() != SOS_token))]
        elif Lang == "L2":
            letters = [self.Lang_To_Index_2_Alpha[letter.item()] for letter in word if ((letter.item() != EOS_token) and (letter.item() != SOS_token))]
        #print([self.EOS_token]*(30-len(indexes)))
        word = ''.join(letters)
        return word
    def PairFromtensors(self,pair):
        input_word = self.WordFromtensors("L1",pair[0])
        target_word = self.WordFromtensors("L2",pair[1])
        return (input_word, target_word)
    '''def DataFromtensors(self,Data):
        Tensors_Val = []
        for i in tqdm(range(Data.shape[0])):
            Tensors_Val.append(self.PairFromtensors(data_train.iloc[i]))
        return Tensors_Val'''
                                             
                                        

In [7]:
T = Tokenize("English","Hindi")
data_train_num = T.tensorsFromData(data_train)
data_val_num = T.tensorsFromData(data_val)
data_test_num = T.tensorsFromData(data_test)
#tensorFromWord(english_alpha2index,data_train_X[0])

100%|██████████| 4096/4096 [00:00<00:00, 5489.39it/s]


In [8]:
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [9]:
train_set=CustomDataset(data_train_num)
valid_set=CustomDataset(data_val_num)
test_set=CustomDataset(data_test_num)

In [10]:
train_data_set=DataLoader(train_set, batch_size=64, shuffle=True)
valid_data_set=DataLoader(valid_set, batch_size=64, shuffle=False)
test_data_set=DataLoader(test_set, batch_size=64, shuffle=False)

800
64
64


# With Attention Encoder-Decoder

In [11]:
class Encoder(nn.Module):
    def __init__(self,input_size,embedding_size,hidden_size,num_layers, dropouts,cell_type,bidirectional):
        super(Encoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropouts)
        self.embedding = nn.Embedding(input_size,embedding_size)
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        if num_layers >1:
            if self.cell_type == "LSTM":
                self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers,dropout=dropouts,bidirectional=self.bidirectional)
            elif self.cell_type == "RNN":
                self.rnn = nn.RNN(embedding_size,hidden_size,num_layers,dropout=dropouts,bidirectional=self.bidirectional)
            elif self.cell_type == "GRU":
                self.rnn = nn.GRU(embedding_size,hidden_size,num_layers,dropout=dropouts,bidirectional=self.bidirectional)
        else:
            if self.cell_type == "LSTM":
                self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers,bidirectional=self.bidirectional)
            elif self.cell_type == "RNN":
                self.rnn = nn.RNN(embedding_size,hidden_size,num_layers,bidirectional=self.bidirectional)
            elif self.cell_type == "GRU":
                self.rnn = nn.GRU(embedding_size,hidden_size,num_layers,bidirectional=self.bidirectional)
                
    def forward(self,x):
        # X : (seq_length,N)
        embedding = self.dropout(self.embedding(x))
        # embedding : seq_length,N,embedding_size)
        if self.cell_type == "LSTM":
            outputs,(hidden,cell) = self.rnn(embedding)
        else:
            outputs,hidden = self.rnn(embedding)
            cell = None
        return outputs,hidden,cell
            

In [12]:
class Decoder(nn.Module):
    def __init__(self,input_size,embedding_size,hidden_size,output_size,num_layers,dropouts,cell_type,bidirectional):
        super(Decoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropouts)
        self.embedding = nn.Embedding(input_size,embedding_size)
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        if num_layers>1:            
            if self.cell_type == "LSTM":
                self.rnn = nn.LSTM((hidden_size*(1+self.bidirectional*1)+embedding_size),hidden_size,num_layers,dropout=dropouts,bidirectional=self.bidirectional)
            elif self.cell_type == "RNN":
                self.rnn = nn.RNN((hidden_size*(1+self.bidirectional*1)+embedding_size),hidden_size,num_layers,dropout=dropouts,bidirectional=self.bidirectional)
            elif self.cell_type == "GRU":
                self.rnn = nn.GRU((hidden_size*(1+self.bidirectional*1)+embedding_size),hidden_size,num_layers,dropout=dropouts,bidirectional=self.bidirectional)
        else:
            if self.cell_type == "LSTM":
                self.rnn = nn.LSTM((hidden_size*(1+self.bidirectional*1)+embedding_size),hidden_size,num_layers,bidirectional=self.bidirectional)
            elif self.cell_type == "RNN":
                self.rnn = nn.RNN((hidden_size*(1+self.bidirectional*1)+embedding_size),hidden_size,num_layers,bidirectional=self.bidirectional)
            elif self.cell_type == "GRU":
                self.rnn = nn.GRU((hidden_size*(1+self.bidirectional*1)+embedding_size),hidden_size,num_layers,bidirectional=self.bidirectional)
        self.energy = nn.Linear(hidden_size*(2+self.bidirectional*1),1)
        self.fc = nn.Linear((1+self.bidirectional*1)*hidden_size,output_size)
        self.weights = nn.Softmax(dim=0)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self,x,encoder_states,hidden,cell):
        # x :(N) but we want (1,N)
        x = x.unsqueeze(0)
        embedding = self.dropout(self.embedding(x))
        # embedding : (1,N,embedding_size)
        sequence_length = encoder_states.shape[0]
        #print("Seq = ",sequence_length)
        #print("Enc = ",encoder_states.shape)
        #print("hidd = ",hidden.shape)
        h_reshaped = hidden.repeat(int(sequence_length/(1+self.bidirectional)),1,1)
        #print(int(sequence_length/(1+self.bidirectional)))
        #print(h_reshaped.shape,encoder_states.shape)
        energy = self.tanh(self.energy(torch.cat((h_reshaped,encoder_states),dim=2)))
        attention_weights = self.weights(energy) 
        # attention : seq_length,N,1
        attention = attention_weights.permute(1,2,0)
        # attention : N,1,seq_length
        encoder_states = encoder_states.permute(1,0,2)
        # (N,1,hidden_size*2) --> (1,N,hidden_size*2)
        context_vector = torch.bmm(attention,encoder_states).permute(1,0,2)
        rnn_input = torch.cat((context_vector,embedding),dim=2)
        
        if self.cell_type == "LSTM":
            outputs,(hidden,cell) = self.rnn(rnn_input,(hidden,cell))
        else:
            outputs,hidden = self.rnn(rnn_input,hidden)
            cell = None
        # outputs : (1,N,hidden_size)
        predictions = self.fc(outputs)
        #predictions : (1,N,output_vocab_size)
        predictions = self.softmax(predictions[0])
        #predictions = predictions.squeeze(0)
        
        return predictions,hidden,cell,attention_weights


In [13]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder):
        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self,source,target,teacher_forceing=0.5):
        batch_size = source.shape[1]
        self.target_len = target.shape[0]
        target_vocab_size = len(hindi_alpha2index)
        
        outputs = torch.zeros(self.target_len,batch_size,target_vocab_size).to(device)
        encoder_states,hidden,cell = self.encoder(source)
        
        # Start Token
        x = target[0]
        for t in range(1,self.target_len):
            output,hidden,cell,_ = self.decoder(x,encoder_states,hidden,cell)
            outputs[t] = output
            best_guess = output.argmax(1)
            x = target[t] if random.random()<teacher_forceing else best_guess
        return outputs
    def predict(self,source,track_attn_weights=False):
        batch_size = source.shape[1]
        target_vocab_size = len(hindi_alpha2index)
        
        outputs = torch.zeros(self.target_len,batch_size,target_vocab_size).to(device)
        encoder_states,hidden,cell = self.encoder(source)
        
        # Start Token
        x = 0*source[0]
        
        if track_attn_weights == False:
            Attention_Weights = None
            for t in range(1,self.target_len):
                output,hidden,cell,_ = self.decoder(x,encoder_states,hidden,cell)
                outputs[t] = output
                best_guess = output.argmax(1)
                x = best_guess
        else:
            Attention_Weights = torch.zeros([batch_size,self.target_len,self.target_len]).to(device)
            for t in range(1,self.target_len):
                output,hidden,cell,attention_weights = self.decoder(x,encoder_states,hidden,cell)
                outputs[t] = output
                best_guess = output.argmax(1)
                x = best_guess
                #print(Attention_Weights.shape,attention_weights.shape)
                Attention_Weights[:,:,t] = attention_weights.permute(1,0,2).squeeze()
            
        return outputs,Attention_Weights
    def find_crct_Tot(self,predicted_batch,target_batch):
        crct,Total=0,0
        for i in range(target_batch.shape[0]):
            Pred = T.WordFromtensors("L2",predicted_batch[i])
            Targ = T.WordFromtensors("L2",target_batch[i])
            Total+=1
            if Pred == Targ:
                crct +=1
        return crct,Total
        
        

In [14]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_validation_Acc = 0

    def early_stop(self, validation_Acc):
        if validation_Acc > self.max_validation_Acc:
            self.max_validation_loss = validation_Acc
            self.counter = 0
        elif validation_Acc < (self.max_validation_Acc + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [15]:
# Training Params
num_epochs = 25
learning_rate = 0.0001
batch_size = 64

# Model Params
input_size_encoder = len(english_alpha2index)
input_size_decoder = len(hindi_alpha2index)
output_size = len(hindi_alpha2index)
encoder_embeddings_size = 512
decoder_embeddings_size = 512
hidden_size = 512
num_layers = 1
enc_dropout = 0.2
dec_dropout = 0.2
cell_type = "LSTM"
bidirectional = True

encoder_net = Encoder(input_size_encoder,encoder_embeddings_size,hidden_size,num_layers,enc_dropout,cell_type,bidirectional).to(device)
decoder_net = Decoder(input_size_decoder,decoder_embeddings_size,hidden_size,output_size,num_layers,dec_dropout,cell_type,bidirectional).to(device)

model=Seq2Seq(encoder_net,decoder_net).to(device)




'''encoder_net = Encoder(input_size_encoder,encoder_embeddings_size,hidden_size,num_layers,enc_dropout).to(device)
decoder_net = Decoder(input_size_decoder,decoder_embeddings_size,hidden_size,output_size,num_layers,dec_dropout).to(device)'''

model = Seq2Seq(encoder_net,decoder_net).to(device)
optimizer = optim.Adam(model.parameters(),lr = learning_rate)
pad_idx = EOS_token
criterion = nn.CrossEntropyLoss()#ignore_index=pad_idx)
Loss_log = []
Max_Acc = 0
early_stopper = EarlyStopper(patience=1, min_delta=0.001)
for epoch in range(num_epochs):
    epoch_loss = 0
    for batch in tqdm(train_data_set):
        inp_data = batch[0].T.to(device)
        target = batch[1].T.to(device)
        #print(inp_data.shape)
        #print(inp_data)
        output = model(inp_data,target)
        #output : (trg_len,batch_size,output_dim)
        output = output[1:].reshape(-1,output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output,target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm = 1)
        optimizer.step()
        epoch_loss += loss.item()
    Loss_log.append(epoch_loss)
    Train_epoch_loss = epoch_loss/len(train_data_set)

    Predictions_List = []
    Total = 0
    crct = 0
    Val_epoch_loss = 0
    for batch in valid_data_set:
        inp_data = batch[0].T.to(device)
        target = batch[1].T.to(device)
        output,_ = model.predict(inp_data)
        #print(output_val[2])
        best_guess = output.argmax(2)
        predictions = best_guess.squeeze()
        #print(predictions.shape)
        output = output[1:].reshape(-1,output.shape[2])
        target = target[1:].reshape(-1)
        loss = criterion(output,target)
        Val_epoch_loss += loss.item()
        for i in range(batch[1].shape[0]):
            Pairs_P = T.PairFromtensors((batch[0][i],predictions.T[i]))
            Pairs_T = T.PairFromtensors((batch[0][i],batch[1][i]))
            Total+=1
            if Pairs_P[1] == Pairs_T[1]:
                crct +=1
    Val_epoch_loss=Val_epoch_loss/len(valid_data_set)
    Val_Accuracy = crct/Total
    print("Epoch = [{}/{}] : Train_loss = {}, val_loss = {}, val_accuracy = {}".format(epoch+1,num_epochs,Train_epoch_loss,Val_epoch_loss,Val_Accuracy))
    if Val_Accuracy>Max_Acc:
        torch.save(model.state_dict(),'ME19B031_Attn_Model.model')
        Model_weights=copy.deepcopy(model.state_dict())
        Max_Acc=Val_Accuracy

100%|██████████| 800/800 [02:08<00:00,  6.21it/s]


Epoch = [1/25] : Train_loss = 0.7150063009187579, val_loss = 0.4713775743730366, val_accuracy = 0.06494140625


100%|██████████| 800/800 [01:35<00:00,  8.42it/s]


Epoch = [2/25] : Train_loss = 0.42464958757162097, val_loss = 0.3790081525221467, val_accuracy = 0.141845703125


100%|██████████| 800/800 [01:59<00:00,  6.70it/s]


...

Epoch = [23/25] : Train_loss = 0.08279922000132502, val_loss = 0.2969949634280056, val_accuracy = 0.39697265625


100%|██████████| 800/800 [02:33<00:00,  5.22it/s]


Epoch = [24/25] : Train_loss = 0.07832737917546183, val_loss = 0.29068760154768825, val_accuracy = 0.395263671875


100%|██████████| 800/800 [02:03<00:00,  6.46it/s]


Epoch = [25/25] : Train_loss = 0.07101300453301519, val_loss = 0.2976457641925663, val_accuracy = 0.406005859375


In [16]:
model.load_state_dict(Model_weights)

<All keys matched successfully>

# Validation Data

In [20]:
Valid_Predictions_List = []
Total = 0
crct = 0
for batch in valid_data_set:
    inp_data = batch[0].T.to(device)
    output_val,_ = model.predict(inp_data)
    #print(output_val[2])
    best_guess = output_val.argmax(2)
    predictions = best_guess.squeeze()
    #print(predictions.shape)
    for i in range(batch[1].shape[0]):
        Pairs_P = T.PairFromtensors((batch[0][i],predictions.T[i]))
        Pairs_T = T.PairFromtensors((batch[0][i],batch[1][i]))
        Total+=1
        if Pairs_P[1] == Pairs_T[1]:
            crct +=1
        Valid_Predictions_List.append([Pairs_T[0],Pairs_T[1],Pairs_P[1]])
print("Validation Accuracy =",crct/Total)

Validation Accuracy = 0.4052734375


In [21]:
dff= pd.DataFrame(np.array(Valid_Predictions_List),columns = ["Source","Target","Predicted"])
dff.to_csv("With_Attention_Valid_Pred.csv", index = False)

# Test Data

In [22]:
Test_Predictions_List = []
Total = 0
crct = 0
for batch in test_data_set:
    inp_data = batch[0].T.to(device)
    output_val,_ = model.predict(inp_data)
    #print(output_val[2])
    best_guess = output_val.argmax(2)
    predictions = best_guess.squeeze()
    #print(predictions.shape)
    for i in range(batch[1].shape[0]):
        Pairs_P = T.PairFromtensors((batch[0][i],predictions.T[i]))
        Pairs_T = T.PairFromtensors((batch[0][i],batch[1][i]))
        Total+=1
        if Pairs_P[1] == Pairs_T[1]:
            crct +=1
        Test_Predictions_List.append([Pairs_T[0],Pairs_T[1],Pairs_P[1]])
print("Test Accuracy =",crct/Total)

Test Accuracy = 0.373779296875


In [23]:
dff= pd.DataFrame(np.array(Test_Predictions_List),columns = ["Source","Target","Predicted"])
dff.to_csv("With_Attention_Test_Pred.csv", index = False)

# Heat Maps

In [17]:
#checkpoint = torch.load("ME19B031_Attn_Model.model")
model.load_state_dict(torch.load("ME19B031_Attn_Model.model"))
#Model_weights = checkpoint.state_dict()
#model.load_state_dict(Model_weights)

<All keys matched successfully>

In [31]:
indx = 1
for batch in test_data_set:
        inp_data = batch[0].T.to(device)
        output_val, Weight = model.predict(inp_data,track_attn_weights=True)
        best_guess = output_val.argmax(2)
        predictions = best_guess.squeeze()
        break

In [20]:
import wandb
wandb.login()

wandb: Currently logged in as: me19b031. Use `wandb login --relogin` to force relogin


True

In [25]:
wandb.init(
      # Set the project where this run will be logged
      project="ME19B031_CS6910_ASSIGNMENT_3_Final", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      #name=f"experiment", 
      # Track hyperparameters and run metadata
      )


#Pairs_P = T.PairFromtensors((batch[0][0],predictions.T[0]))
Mat=Weight[-1].cpu().detach().numpy()[1:len(Pairs_P[1])+1,1:len(Pairs_P[0])+1]
wandb.log({"{}".format(Pairs_P[0]): wandb.plots.HeatMap(list(Pairs_P[0]), list(Pairs_P[1]), Mat, show_text=False)})

wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Visualizing heatmap.


In [33]:
for i in range(20):
    Pairs_P = T.PairFromtensors((batch[0][i],predictions.T[i]))
    #Pairs_P = T.PairFromtensors((batch[0][0],predictions.T[0]))
    Mat=Weight[i].cpu().detach().numpy()#[1:len(Pairs_P[1])+1,1:len(Pairs_P[0])+1]
    X_label = 
    wandb.log({"{}".format(Pairs_P[0]): wandb.plots.HeatMap(list(Pairs_P[0]), list(Pairs_P[1]), Mat, show_text=False)})

wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
